In [1]:
import os

In [2]:
%pwd

'c:\\Users\\USER\\Documents\\JupyterNB\\ineuron\\Python Advanced\\Phishing_detection\\notebooks'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\USER\\Documents\\JupyterNB\\ineuron\\Python Advanced\\Phishing_detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelBuildingConfig:
    root_dir: Path
    trained_modl_file: Path
    LogisticRegression: dict
  




In [6]:
from src.phishingdetection.constants import *
from src.phishingdetection.utils.common_functionality import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_building_config(self) -> ModelBuildingConfig:
        config = self.config.model_building

        create_directories([config.root_dir])

        model_building_config = ModelBuildingConfig(
            root_dir = config.root_dir,
            trained_modl_file = config.trained_modl_file,
            LogisticRegression = self.params.LogisticRegression,
       
            
        )

        return model_building_config

In [8]:
from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from src.phishingdetection.utils.common_functionality import load_object, save_object, get_size
from src.phishingdetection import logger
import pandas as pd

In [9]:
class ModelBuilding:
    def __init__(self, config: ModelBuildingConfig):
        self.config = config


    def train_model(self,final_data_path):
        
            
        final_data_for_training = pd.read_pickle(final_data_path)

        X_train = final_data_for_training['X_train']
        y_train = final_data_for_training['y_train']
  
        
        random_src = RandomizedSearchCV(estimator= LogisticRegression(), param_distributions=read_yaml(PARAMS_FILE_PATH).to_dict()['LogisticRegression'], cv=3, n_jobs=-1)
        random_src.fit(X_train,y_train)
        print(random_src.best_estimator_)
        print(random_src.best_score_)
        print(random_src.best_params_)
            
     


        # model = GradientBoostingClassifier(criterion='squared_error', loss='exponential', n_estimators=200)

        # model.fit(X_train,y_train)

        # if not os.path.exists(self.config.trained_modl_file):
        #     save_object(self.config.trained_modl_file,model)
        #     logger.info('Trained model is saved')
        # else:
        #     logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

            
            


In [10]:
try:
    config = ConfigurationManager()
    
    model_building_config = config.get_model_building_config()
    model_building = ModelBuilding(config=model_building_config)
    model_building.train_model(config.config.data_processing.final_data_file)
   
except Exception as e:
    raise e

[2023-07-24 15:49:45,848: INFO: common_functionality: yaml file: config\config.yaml loaded successfully]
[2023-07-24 15:49:45,876: INFO: common_functionality: yaml file: params.yaml loaded successfully]
[2023-07-24 15:49:45,879: INFO: common_functionality: created directory at: artifacts]
[2023-07-24 15:49:45,883: INFO: common_functionality: created directory at: artifacts/model_building]


[2023-07-24 15:49:45,986: INFO: common_functionality: yaml file: params.yaml loaded successfully]
LogisticRegression(C=0.01)
0.8954691842812249
{'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.01}
